In [1]:
import pandas as pd 
import numpy as np 
from datetime import datetime, timedelta
import math
import re

In [122]:
data = pd.read_csv('test.csv')

In [123]:
data.pop('Unnamed: 0')
xx = data['Salary'].value_counts()
y= xx.pop('notfound')
x = xx.sum()
xxx = data['Salary'].value_counts().sum()

In [124]:
print("J'ai " + str(x) + " Valeurs de salaires pour " + str(y) + " Na , avec un total de " + str(xxx))
x*100 /xxx


J'ai 726 Valeurs de salaires pour 3673 Na , avec un total de 4399


16.50375085246647

- Drop les doublons : 

In [2]:
def getdata(urlfiles):
    data = pd.read_csv(urlfiles)
    data.pop('Unnamed: 0')
    data = data.drop_duplicates(subset =['Title', 'Company', 'Location', 'Salary', 'Description','TypeContrat'], keep= 'first',inplace=False)
    return data
data = getdata('test.csv')
xx = data['Salary'].value_counts()
y= xx.pop('notfound')
x = xx.sum()
xxx = data['Salary'].value_counts().sum()
print("J'ai " + str(x) + " Valeurs de salaires pour " + str(y) + " Na , avec un total de " + str(xxx))

J'ai 231 Valeurs de salaires pour 1309 Na , avec un total de 1540


- Comment Catégoriser les dates : 

In [3]:
def MakeDateGreatAgain(df):
    dede = []
    for k,i in df.iterrows() : 
        if i['Date'].lower().replace(' ','') == "publiéeàl'instant" or i['Date'].lower().replace(' ','') == "aujourd'hui":
            d = datetime.today()
            df['Date'][k] = d
        elif i['Date'].lower().replace(' ','') == "ilya30+jours":
            d = datetime.today() - timedelta(days=31)
            df['Date'][k] = d
        else: 
            if len(i['Date'].split()) == 5 :
                if i['Date'].split()[3].isdigit():
                    d = datetime.today() - timedelta(days=int(i['Date'].split()[3]))
                    df['Date'][k] = d
                else : 
                    df.drop(k)
            else :
                dede.append(k)
    return df.drop(dede)

data = MakeDateGreatAgain(data)

- Definir les types de contrats

In [4]:
def Makedummiestypecontrat(data):
    doto = pd.DataFrame(columns= ['TypeContrat_CDI','TypeContrat_Temps plein','TypeContrat_Apprentissage','TypeContrat_Contrat pro','TypeContrat_CDD','TypeContrat_Intérim','TypeContrat_Temps partiel','TypeContrat_Stage','TypeContrat_Indépendant','TypeContrat_notfound'])
    for key ,row in data.iterrows():
        doto.loc[key]= 0
        for typecont in row['TypeContrat'].split(','):
            if typecont.strip() == 'Freelance / Indépendant':
                namecol = 'TypeContrat_'+ 'Indépendant'
                doto[namecol][key] =1
            else :
                namecol = 'TypeContrat_'+ typecont.strip()
                doto[namecol][key] =1
                
    data = pd.concat([data, doto],axis=1)
    data.drop(['TypeContrat_notfound'],axis=1,inplace=True)
    return data
       
data = Makedummiestypecontrat(data)

- définir la moyenne des salaires

In [5]:
def getmeansalary(df):
    for key ,row in df.iterrows():
        if row['Salary'] == "notfound":
            df['Salary'][key] = 'notfound'
        else: 
            non_decimal = re.compile(r'[^\d.]+')
            separatenum = str(row['Salary']).split('-')
            if len(separatenum) == 1 :
                if 'par an' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)
                elif 'par mois' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)*12
                elif 'par jour' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)*22*12
                elif 'par semaine' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)*4*12
                elif 'par heure' in separatenum[0]:
                    justeprix= non_decimal.sub('', separatenum[0])
                    df['Salary'][key] = int(justeprix)*8*22*12
            else: 
                if 'par an' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche),int(hauttranche)])
                    df['Salary'][key] = moyennealannee
                elif 'par mois' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche)*12,int(hauttranche)*12])
                    df['Salary'][key] = moyennealannee
                elif 'par jour' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche)*22*12,int(hauttranche)*22*12])
                    df['Salary'][key] = moyennealannee
                elif 'par semaine' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche)*4*12,int(hauttranche)*4*12])
                    df['Salary'][key] = moyennealannee
                elif 'par heure' in separatenum[1]:
                    bastranche= non_decimal.sub('', separatenum[0])
                    hauttranche= non_decimal.sub('', separatenum[1])
                    moyennealannee= np.mean([int(bastranche)*8*22*12,int(hauttranche)*8*22*12])
                    df['Salary'][key] = moyennealannee
                else: 
                    print("Error : Tout n'est pas passé ")
    
    return df
                

data = getmeansalary(data)

- gestion des localisations 

In [66]:
def GetcategLocation(df):
    for key, row in df.iterrows():
        if '75' in row['Location'] or '92' in row['Location'] or '94' in row['Location'] or '93' in row['Location'] or '78' in row['Location'] or '95' in row['Location'] or '77' in row['Location'] or '91' in row['Location'] or 'Hauts-de-Seine'in row['Location'] or 'Val-de-Marne' in row['Location'] or 'Île-de-France' in row['Location']  :
            df['Location'][key] = 'Paris'
        elif '69' in row['Location'] or 'Auvergne-Rhône-Alpes' in row['Location'] or 'Occitanie' in row['Location'] or 'Rhône' in row['Location'] or '38' in row['Location'] :
            df['Location'][key] = 'Lyon'
        elif '54' in row['Location'] or "57" in row['Location'] or 'Meurthe-et-Moselle' in row['Location'] or '01'in row['Location'] or 'Meurthe-et-Moselle'in row['Location'] or 'Grand Est' in row['Location']:
            df['Location'][key] = 'Nancy'
        elif '33' in row['Location'] or 'Gironde' in row['Location'] or 'Haute-Garonne' in row['Location'] :
            df['Location'][key] = 'Bordeaux'
        elif '13' in row['Location'] or "Provence-Alpes-Côte d'Azur" in row['Location']:
            df['Location'][key] = 'Marseille'
        elif '31' in row['Location']  or 'Nouvelle-Aquitaine' in row['Location']  :
            df['Location'][key] = 'Toulouse'
        elif '44' in row['Location']  or 'Pays de la Loire' in row['Location'] or 'Loire-Atlantique' in row['Location']:
            df['Location'][key] = 'Nantes'
        else : 
            df['Location'][key] = row['Location']
    return df
dutu = GetcategLocation(dutu)
dutu['Location'].unique()

array(['Paris', 'Bordeaux', 'France', 'Lyon', 'Nancy', 'Marseille',
       'Toulouse', 'Nantes'], dtype=object)

In [84]:
def getpreprocessdata(df):
    df = MakeDateGreatAgain(df)
    df = Makedummiestypecontrat(df)
    df= getmeansalary(df)
    df = GetcategLocation(df)
    return df


data = getdata('test.csv')
data= getpreprocessdata(data)

In [76]:
doto= pd.read_csv('data/3500rows+jobville.csv')
doto = doto.drop(['Job','Ville','Unnamed: 0'],axis=1)
doto = doto.drop_duplicates(subset =['Title', 'Company', 'Location', 'Salary', 'Description','TypeContrat'], keep= 'first',inplace=False)
doto = getpreprocessdata(doto)
doto

,Title,Company,Location,TypeContrat,Salary,Description,Date,Link,TypeContrat_CDI,TypeContrat_Temps plein,TypeContrat_Apprentissage,TypeContrat_Contrat pro,TypeContrat_CDD,TypeContrat_Intérim,TypeContrat_Temps partiel,TypeContrat_Stage,TypeContrat_Indépendant
0,Alternance - Data Scientist H/F,Malakoff Humanis,Paris,"Apprentissage, Contrat pro",notfound,Contrat\r\nALTERNANCE\r\nFinalité du poste\r\n...,2020-05-16 14:11:39.382101,/pagead/clk?mo=r&ad=-6NYlbfkN0BfLf6vlT6l4ABp3X...,0,0,1,1,0,0,0,0,0
1,Data Scientist (F/H),Novencia,Paris,notfound,notfound,Contexte\r\nData is fuel ! Quelle que soit la ...,2020-04-19 14:11:39.382101,/pagead/clk?mo=r&ad=-6NYlbfkN0Daqc6iXyKFbG6VVD...,0,0,0,0,0,0,0,0,0
2,Commercial / Sales,Realist,Paris,"Temps plein, CDI",notfound,Commercial / Sales\r\nEntreprise\r\nLa mission...,2020-05-16 14:11:39.382101,/pagead/clk?mo=r&ad=-6NYlbfkN0B7k0JWKsIdrfTyIz...,1,1,0,0,0,0,0,0,0
3,Développeur Ruby / React JS - Paris 3 - 45/55k€,Pearson Frank,Paris,notfound,55000,Notre client propose des solutions sur mesure ...,2020-05-18 14:11:39.382101,/pagead/clk?mo=r&ad=-6NYlbfkN0Bl_GJvjV7T_nyT7Y...,0,0,0,0,0,0,0,0,0
4,DATA SCIENTIST / MACHINE LEARNING RESEARCHER I...,Joko,Paris,Stage,notfound,DESCRIPTIF DU POSTE\r\n\r\nTu seras responsabl...,2020-05-16 14:11:39.383114,/rc/clk?jk=51c591314452c91c&fccid=0b5f93d5f9c5...,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3375,Contrôleur de Gestion H/F,Vidal Associates,Marseille,notfound,notfound,"VIDAL ASSOCIATES Consulting & Search, référenc...",2020-04-19 14:11:39.595116,/rc/clk?jk=95a42514723d392a&fccid=6ffe4b5e38e3...,0,0,0,0,0,0,0,0,0
3376,Ingénieur développement F/H,AtmoSud,Marseille,notfound,notfound,Votre rôle consistera à suivre les développeme...,2020-04-25 14:11:39.595116,/rc/clk?jk=0b08db3cc6dae4df&fccid=ef13f183d8a2...,0,0,0,0,0,0,0,0,0
3378,Trésorier F/H - anglais courant Marseille,Fed Finance,Marseille,CDI,notfound,Votre fonction\r\n\r\nRattaché(e) au Directeur...,2020-04-23 14:11:39.595116,/rc/clk?jk=d7b5dc07c6b32699&fccid=65d13b9da141...,1,0,0,0,0,0,0,0,0
3379,Consultant Data Visualisation H/F (Aix-en-Prov...,Keyrus,Marseille,CDI,notfound,Qui sommes-nous ?\r\nUne success story dans la...,2020-04-19 14:11:39.596110,/rc/clk?jk=f638e98a0c845072&fccid=19a55a04a564...,1,0,0,0,0,0,0,0,0


In [77]:
dete= pd.read_csv('data/1185rows.csv')
dete = dete.drop('Unnamed: 0',axis= 1)
dete = getpreprocessdata(dete)
dete

,Title,Company,Location,TypeContrat,Salary,Description,Date,Link,TypeContrat_CDI,TypeContrat_Temps plein,TypeContrat_Apprentissage,TypeContrat_Contrat pro,TypeContrat_CDD,TypeContrat_Intérim,TypeContrat_Temps partiel,TypeContrat_Stage,TypeContrat_Indépendant
0,Alternance - Data Scientist H/F,Malakoff Humanis,Paris,"Apprentissage, Contrat pro",notfound,Contrat\r\nALTERNANCE\r\nFinalité du poste\r\n...,2020-05-17 14:11:41.498279,/pagead/clk?mo=r&ad=-6NYlbfkN0BfLf6vlT6l4ABp3X...,0,0,1,1,0,0,0,0,0
1,DATA analyste informatique (F/H),ACCEO CONSULTING,Paris,notfound,notfound,"Actuellement, au sein des équipes BIG DATA et ...",2020-05-03 14:11:41.499280,/pagead/clk?mo=r&ad=-6NYlbfkN0AHHRM07jvZ_9CBb8...,0,0,0,0,0,0,0,0,0
2,Chef de Projet Data F/H,TRIMANE,Paris,notfound,notfound,TRIMANE est une société de service spécialisée...,2020-05-10 14:11:41.499280,/pagead/clk?mo=r&ad=-6NYlbfkN0Bl_GJvjV7T_nyT7Y...,0,0,0,0,0,0,0,0,0
3,Data Scientist H/F,NEOBRAIN,Paris,"Temps plein, CDI",notfound,Neobrain développe des solutions digitales int...,2020-05-20 14:11:41.499280,/company/NEOBRAIN/jobs/Data-Scientist-dfce81a4...,1,1,0,0,0,0,0,0,0
4,Stagiaire Data Scientist H/F,Cocoparks,Paris,"Temps plein, Stage",notfound,Description de l'entrepriseCocoparks is an ear...,2020-05-18 14:11:41.499280,/company/Cocoparks/jobs/Stagiaire-Data-Scienti...,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180,ANALYSTE EN INFORMATIQUE INDUSTRIELLE H/F,Proman interim,Bordeaux,Intérim,notfound,Votre agence PROMAN BORDEAUX RECRUTEMENT reche...,2020-05-16 14:11:41.755124,/rc/clk?jk=3608aa8e53dea31f&fccid=d2d18ddd5178...,0,0,0,0,0,1,0,0,0
1181,Leader Technique BI H/F (Bordeaux),Keyrus,Bordeaux,CDI,notfound,Qui sommes-nous ?\r\nUne success story dans la...,2020-04-19 14:11:41.756111,/rc/clk?jk=5639f3c3fb2004bc&fccid=19a55a04a564...,1,0,0,0,0,0,0,0,0
1182,ANALYSTE EN INFORMATIQUE INDUSTRIELLE H/F,PROMAN BORDEAUX RECRUTEMENT,Bordeaux,"Temps plein, Intérim",notfound,PROMAN est un acteur majeur dans le domaine de...,2020-05-16 14:11:41.756111,/rc/clk?jk=2c6c24c36f6014c5&fccid=7d8403e39250...,0,1,0,0,0,1,0,0,0
1183,"Consultant Java, J2EE H/F (Pau)",Keyrus,Bordeaux,CDI,notfound,Qui sommes-nous ?\r\nUne success story dans la...,2020-04-19 14:11:41.756111,/rc/clk?jk=78a40eeae476bb35&fccid=19a55a04a564...,1,0,0,0,0,0,0,0,0


In [78]:
dyty= pd.read_csv('data/1030entree.csv')
dyty = dyty.drop('Unnamed: 0',axis=1)
dyty = getpreprocessdata(dyty)
dyty

,Title,Company,Location,TypeContrat,Salary,Description,Date,Link,TypeContrat_CDI,TypeContrat_Temps plein,TypeContrat_Apprentissage,TypeContrat_Contrat pro,TypeContrat_CDD,TypeContrat_Intérim,TypeContrat_Temps partiel,TypeContrat_Stage,TypeContrat_Indépendant
0,Alternance - Data Scientist H/F,Malakoff Humanis,Paris,"Apprentissage, Contrat pro",notfound,Contrat\r\nALTERNANCE\r\nFinalité du poste\r\n...,2020-05-17 14:11:43.862175,/pagead/clk?mo=r&ad=-6NYlbfkN0BfLf6vlT6l4ABp3X...,0,0,1,1,0,0,0,0,0
1,Data Scientist (F/H),Novencia,Paris,notfound,notfound,Contexte\r\nData is fuel ! Quelle que soit la ...,2020-04-19 14:11:43.862175,/pagead/clk?mo=r&ad=-6NYlbfkN0Daqc6iXyKFbG6VVD...,0,0,0,0,0,0,0,0,0
2,Data Scientist,DEGETEL,Paris,notfound,notfound,Degetel ? Degetel et le digital ont en commun ...,2020-04-19 14:11:43.862175,/pagead/clk?mo=r&ad=-6NYlbfkN0Bl_GJvjV7T_nyT7Y...,0,0,0,0,0,0,0,0,0
3,DATA SCIENTIST / MACHINE LEARNING RESEARCHER I...,Joko,Paris,Stage,notfound,DESCRIPTIF DU POSTE\r\n\r\nTu seras responsabl...,2020-05-17 14:11:43.862175,/rc/clk?jk=51c591314452c91c&fccid=0b5f93d5f9c5...,0,0,0,0,0,0,0,1,0
4,Data Scientist H/F,NEOBRAIN,Paris,"Temps plein, CDI",notfound,Neobrain développe des solutions digitales int...,2020-05-20 14:11:43.863174,/company/NEOBRAIN/jobs/Data-Scientist-dfce81a4...,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,INTÉGRATEUR HTML/CSS/JS (F/H),SQLI,Lyon,CDI,notfound,"A propos de SQLI\r\nCréée en 1990, SQLI est un...",2020-04-19 14:11:44.085593,/pagead/clk?mo=r&ad=-6NYlbfkN0AHJPUinF6US8hAwh...,1,0,0,0,0,0,0,0,0
1026,Développeur Full Stack,klervi,Lyon,CDI,notfound,À propos de klerviLa société klervi est un bur...,2020-04-19 14:11:44.085593,/pagead/clk?mo=r&ad=-6NYlbfkN0D6vbzA83nncZIUYV...,1,0,0,0,0,0,0,0,0
1027,Développeur .NET Junior H/F,Berger-Levrault,Lyon,CDI,notfound,BERGER-LEVRAULT est éditeur de logiciels inter...,2020-04-19 14:11:44.086606,/pagead/clk?mo=r&ad=-6NYlbfkN0BMIloTkKDvAIDiY1...,1,0,0,0,0,0,0,0,0
1028,(ALTERNANCE) Développeur Web - Chez l’un de no...,OpenClassrooms,France,"Temps plein, Apprentissage",notfound,La formation « Développeur web » en alternance...,2020-04-19 14:11:44.086606,/pagead/clk?mo=r&ad=-6NYlbfkN0Ca2Y14vJUI4dcWvj...,0,1,1,0,0,0,0,0,0


In [79]:
diti = pd.concat([doto,dete,dyty],axis=0)
diti = diti.drop_duplicates(subset =['Title', 'Company', 'Location', 'Salary', 'Description','TypeContrat'], keep= 'first',inplace=False)
xx = diti['Salary'].value_counts()
y= xx.pop('notfound')
x = xx.sum()
xxx = diti['Salary'].value_counts().sum()
print("J'ai " + str(x) + " Valeurs de salaires pour " + str(y) + " Na , avec un total de " + str(xxx))
diti

J'ai 223 Valeurs de salaires pour 1260 Na , avec un total de 1483


,Title,Company,Location,TypeContrat,Salary,Description,Date,Link,TypeContrat_CDI,TypeContrat_Temps plein,TypeContrat_Apprentissage,TypeContrat_Contrat pro,TypeContrat_CDD,TypeContrat_Intérim,TypeContrat_Temps partiel,TypeContrat_Stage,TypeContrat_Indépendant
0,Alternance - Data Scientist H/F,Malakoff Humanis,Paris,"Apprentissage, Contrat pro",notfound,Contrat\r\nALTERNANCE\r\nFinalité du poste\r\n...,2020-05-16 14:11:39.382101,/pagead/clk?mo=r&ad=-6NYlbfkN0BfLf6vlT6l4ABp3X...,0,0,1,1,0,0,0,0,0
1,Data Scientist (F/H),Novencia,Paris,notfound,notfound,Contexte\r\nData is fuel ! Quelle que soit la ...,2020-04-19 14:11:39.382101,/pagead/clk?mo=r&ad=-6NYlbfkN0Daqc6iXyKFbG6VVD...,0,0,0,0,0,0,0,0,0
2,Commercial / Sales,Realist,Paris,"Temps plein, CDI",notfound,Commercial / Sales\r\nEntreprise\r\nLa mission...,2020-05-16 14:11:39.382101,/pagead/clk?mo=r&ad=-6NYlbfkN0B7k0JWKsIdrfTyIz...,1,1,0,0,0,0,0,0,0
3,Développeur Ruby / React JS - Paris 3 - 45/55k€,Pearson Frank,Paris,notfound,55000,Notre client propose des solutions sur mesure ...,2020-05-18 14:11:39.382101,/pagead/clk?mo=r&ad=-6NYlbfkN0Bl_GJvjV7T_nyT7Y...,0,0,0,0,0,0,0,0,0
4,DATA SCIENTIST / MACHINE LEARNING RESEARCHER I...,Joko,Paris,Stage,notfound,DESCRIPTIF DU POSTE\r\n\r\nTu seras responsabl...,2020-05-16 14:11:39.383114,/rc/clk?jk=51c591314452c91c&fccid=0b5f93d5f9c5...,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,Développeur Web Microsoft (H/F),Exakis,Lyon,CDI,notfound,"Exakis Nelite, entité du groupe Magellan Partn...",2020-04-19 14:11:44.084608,/rc/clk?jk=2492975828792445&fccid=74b179ae010f...,1,0,0,0,0,0,0,0,0
1021,Développeur PHP (H/F),Apside,Lyon,notfound,notfound,Et si vous étiez prêt pour un challenge qui so...,2020-04-19 14:11:44.084608,/rc/clk?jk=5a58da742f83dea9&fccid=c4b1242d6123...,0,0,0,0,0,0,0,0,0
1022,ANALYSTE DÉVELOPPEUR C# - APPLICATIONS WEB (H/F),Crédit mutuel,Lyon,CDI,notfound,Les équipes « Ergonomie et Services Distants »...,2020-04-19 14:11:44.085593,/rc/clk?jk=7583f6ab09cd15e3&fccid=064584719361...,1,0,0,0,0,0,0,0,0
1023,Développeur Android (Java ou Kotlin) - AXOPEN ...,AXOPEN,Lyon,"Temps plein, CDI",notfound,"Pour toi, Android est le meilleur OS de tous l...",2020-05-13 14:11:44.085593,/company/AXOPEN/jobs/D%C3%A9veloppeur-Android-...,1,1,0,0,0,0,0,0,0


In [85]:
dutu = pd.concat([data,diti],axis=0)
dutu = dutu.drop_duplicates(subset =['Title', 'Company', 'Location', 'Salary', 'Description','TypeContrat'], keep= 'first',inplace=False)
xx = dutu['Salary'].value_counts()
y= xx.pop('notfound')
x = xx.sum()
xxx = dutu['Salary'].value_counts().sum()
print("J'ai " + str(x) + " Valeurs de salaires pour " + str(y) + " Na , avec un total de " + str(xxx))
dutu

J'ai 346 Valeurs de salaires pour 1842 Na , avec un total de 2188


,Title,Company,Location,TypeContrat,Salary,Description,Date,Link,TypeContrat_CDI,TypeContrat_Temps plein,TypeContrat_Apprentissage,TypeContrat_Contrat pro,TypeContrat_CDD,TypeContrat_Intérim,TypeContrat_Temps partiel,TypeContrat_Stage,TypeContrat_Indépendant
0,Alternance - Data Scientist H/F,Malakoff Humanis,Paris,"Apprentissage, Contrat pro",notfound,Contrat\r\nALTERNANCE\r\nFinalité du poste\r\n...,2020-05-16 14:12:50.757331,/pagead/clk?mo=r&ad=-6NYlbfkN0BfLf6vlT6l4ABp3X...,0,0,1,1,0,0,0,0,0
1,FULL REMOTE Senior Data Scientist H/F - CDI,Jellysmack,Paris,"Temps plein, CDI",notfound,Nous continuons de recruter et avons adapté no...,2020-04-25 14:12:50.758330,/pagead/clk?mo=r&ad=-6NYlbfkN0B8n3TtewkfrSQLVL...,1,1,0,0,0,0,0,0,0
2,Chef de projets ERP,Castelis,Paris,notfound,50000,"Présentation de la société Castelis conçoit, d...",2020-04-20 14:12:50.758330,/pagead/clk?mo=r&ad=-6NYlbfkN0Bl_GJvjV7T_nyT7Y...,0,0,0,0,0,0,0,0,0
3,ASSISTANT DATA SCIENTIST - STAGE 3 MOIS H/F,BNP Paribas,Paris,"Temps plein, Stage",notfound,ASSISTANT DATA SCIENTIST - STAGE 3 MOIS H/F (N...,2020-05-20 14:12:50.758330,/rc/clk?jk=dc84057ced14dfad&fccid=1392500f835b...,0,1,0,0,0,0,0,1,0
4,Data Scientist H/F,NEOBRAIN,Paris,"Temps plein, CDI",notfound,Neobrain développe des solutions digitales int...,2020-05-20 14:12:50.758330,/company/NEOBRAIN/jobs/Data-Scientist-dfce81a4...,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,Développeur Web Microsoft (H/F),Exakis,Lyon,CDI,notfound,"Exakis Nelite, entité du groupe Magellan Partn...",2020-04-19 14:11:44.084608,/rc/clk?jk=2492975828792445&fccid=74b179ae010f...,1,0,0,0,0,0,0,0,0
1021,Développeur PHP (H/F),Apside,Lyon,notfound,notfound,Et si vous étiez prêt pour un challenge qui so...,2020-04-19 14:11:44.084608,/rc/clk?jk=5a58da742f83dea9&fccid=c4b1242d6123...,0,0,0,0,0,0,0,0,0
1022,ANALYSTE DÉVELOPPEUR C# - APPLICATIONS WEB (H/F),Crédit mutuel,Lyon,CDI,notfound,Les équipes « Ergonomie et Services Distants »...,2020-04-19 14:11:44.085593,/rc/clk?jk=7583f6ab09cd15e3&fccid=064584719361...,1,0,0,0,0,0,0,0,0
1023,Développeur Android (Java ou Kotlin) - AXOPEN ...,AXOPEN,Lyon,"Temps plein, CDI",notfound,"Pour toi, Android est le meilleur OS de tous l...",2020-05-13 14:11:44.085593,/company/AXOPEN/jobs/D%C3%A9veloppeur-Android-...,1,1,0,0,0,0,0,0,0


In [30]:
from Connection import insertDFtoMongo
insertDFtoMongo(dutu,"reunis")

In [31]:
dutu.to_csv(r'C:\Users\Utilisateur\Videos\Python\semaine18distance9projet3\indeed\data\reuni.csv')

- categoriser lexperience grace au descirptions

In [131]:
i=0
j= 0
k=0
for key, row in data.iterrows() : 
    if 'junior' in row['Description'].lower():
        i+=1
    elif 'senior' in row['Description'].lower():
        k+=1
    elif 'junior' in row['Title'].lower():
        i+=1
    elif 'senior' in row['Title'].lower():
        k+=1
    else : 
        for phrase in row['Description'].strip('.') :
            if 'Experiences:' in phrase.strip():
                 j+=1
print(i, 'junior allowed -', k, 'Senior needed -', j , 'notfound')

95 junior allowed - 73 Senior needed - 0 notfound


- Comment catégoriser les noms de Job, pour affiner les prédictions : {EN COURS}

In [90]:
tmp = data['Title'].unique()
bla = []
bla2 = []
i=0
for key, row in dutu.iterrows() : 
    if 'data scientist' in row['Title'].lower() or 'datascientist' in row['Title'].lower() or 'scientist' in row['Title'].lower():
        bla.append('Data Scientist')
    elif 'machine learning' in row['Title'].lower() or 'artificial' in row['Title'].lower() or 'ia' in row['Title'].lower(): 
        bla.append('Machine Learning')
    elif 'data analyst' in row['Title'].lower() or 'analyste' in row['Title'].lower() or 'analyst' in row['Title'].lower(): 
        bla.append('Data Analyst')
    elif 'data engineer' in row['Title'].lower() or 'ingénieur data' in row['Title'].lower() or 'data ingénieur' in row['Title'].lower(): 
        bla.append('Data Engineer')
    elif 'big data' in row['Title'].lower() or 'cloud' in row['Title'].lower() : 
        bla.append('Big Data')
    elif 'manager' in row['Title'].lower() : 
        bla.append('Manager')
    elif 'security' in row['Title'].lower() or 'sécurité' in row['Title'].lower() : 
        bla.append('Cyber sécurity')
    elif 'consultant' in row['Title'].lower() : 
        bla.append('Consultant')
    elif 'data' in row['Title'].lower() or 'bi' in row['Title'].lower() or 'business inteligence' in row['Title'].lower() or 'business developper' in row['Title'].lower() : 
        bla.append('Data')
    elif 'ingenieur' in row['Title'].lower() or 'ingénieur' in row['Title'].lower() or 'engineer' in row['Title'].lower(): 
        bla.append('Ingénieur')
    elif 'web' in row['Title'].lower() or 'fullstack'in row['Title'].lower().replace('-','').replace(' ','') or 'intégrateur' in row['Title'].lower() : 
        bla.append('Dev Web')
    elif 'web' in row['Title'].lower() or 'frontend'in row['Title'].lower().replace('-','').replace(' ','') or 'javascript'in row['Title'].lower() : 
        bla.append('Dev Front end')
    elif 'php'in row['Title'].lower() or  'backend'in row['Title'].lower().replace('-','').replace(' ','')  : 
        bla.append('Dev Back end')
    elif 'développeur' in row['Title'].lower() or 'developpeur' in row['Title'].lower() or 'développeu' in row['Title'].lower() or 'developer' in row['Title'].lower() or 'dev' in row['Title'].lower(): 
        bla.append('developpeur')
    else : 
        bla.append(row['Title'])
d = pd.Series(bla).value_counts()
d

developpeur                                                                                             311
Data Analyst                                                                                            297
Dev Web                                                                                                 257
Data Scientist                                                                                          217
Data Engineer                                                                                           178
                                                                                                       ... 
Conseiller.ère Clientèle - Agence de Croix-Rousse (69)                                                    1
ALTERNANT(E) CHARGE D’ETUDES MARKETING                                                                    1
Livreur à vélo H/F                                                                                        1
Administrateur•trice des sys

In [87]:
347*100/2190

15.844748858447488